<a href="https://colab.research.google.com/github/xuanthang482/CS114.K21_CourseProject/blob/master/ML_helmet_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/mydrive', True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mydrive


In [ ]:
import cv2
import sys
from google.colab import files
from google.colab.patches import cv2_imshow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io
from skimage.io import imread, imshow
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import os
import random
from sklearn.metrics import confusion_matrix

In [ ]:
path = '/mydrive/My Drive/ML_Project/Train_helmet/'
filenames=os.listdir(path)

Các image lưu trong drive dưới dạng 1.+(...).jpg biêu thị cho image có người đội mũ . 0.+(...).jpg biểu thị cho ảnh người không đội mũ . Em dùng 1 mảng chứa các label của ảnh 

In [ ]:
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='1':
        categories.append('1')
    else:
        categories.append('0')

In [ ]:
image = load_img(path+filenames[0])
plt.imshow(image)

# **Tiền xử lí dữ liệu**

In [ ]:
def crop()
  for imgpath in os.listdir(path):
    #resize về kích thước 224,224 và cắt ảnh chỉ lấy nữa trên của ảnh
    im = load_img(path+"/"+imgpath,target_size=(224,224))
    width, height = im.size
    xx = width*0.2
    yy =0
    w = width*0.9
    h = height*0.50
    im1 =im.crop((xx, yy, w, h))
    im1= np.array (im1)
    #làm mờ ảnh  
    im2 = cv2.GaussianBlur(im1, (3, 3), 0)
    #sử dụng canny edge để tìm ra các cạnh có trong ảnh
    a = cv2.Canny(im2, 150, 200)
    a=cv2.resize(a,(200,200))
    #list f bao gồm n ảnh đã qua xử lí có dạng là (n,224,224)

Đây là ảnh trước và sau khi xử lí

In [ ]:
image = load_img(path+filenames[500])
plt.imshow(image)
cv2_imshow(f[500])

# Feature Extraction with canny edge 

Sau khi ảnh được đưa về dạng trắng đen(đã xử lí bằng canny) thì để rút ra được đặt trưng của ảnh em dùng cách tính độ lệch chuân

In [ ]:

def calculate(img):
    # tạo mảng trống có kiểu uint8
    features = np.array([], dtype='uint8')
    section = 1
    for y in range(0, img.shape[0], 40):
        for x in range(0, img.shape[1], 40):

            # Cắt hình ảnh thành một phần 40x40
            section_img = img[y:y+40, x:x+40] 
            # Tính giá trị trung bình và tiêu chuẩn của hình ảnh được phân chia.
            section_mean = np.mean(section_img)
            section_std = np.std(section_img)
            # Thêm các giá trị vừa tính vào mảng
            features = np.append(features, [section_mean, section_std])
    
    # Returning mảng
    return features

In [ ]:
s=calculate(f[0])
s.shape

In [ ]:
def df_created(filesname,category):
  feature_names = []
  section = 1  
  for y in range(0, 200, 40):
    for x in range(0, 200, 40):
        feature_names.append(f"sec{section}_mean")
        feature_names.append(f"sec{section}_std")
        section += 1
  all_imgs = np.zeros((1, 50), dtype='uint8')
  #all_imgs = np.append(all_imgs, img_features, axis=0)
  for i in range(len(filesname)):
      img_features = calculate(filesname[i])
      
      img_features = img_features.reshape(1, img_features.shape[0])

      all_imgs = np.append(all_imgs, img_features, axis=0)

 all_imgs = all_imgs[1:]

  df = pd.DataFrame(all_imgs, columns=feature_names)
  df.insert(loc=50, column="name", value=category)
  return df

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(categories)
labels

In [ ]:
feature_df=df_created(f,labels)

In [ ]:
feature_names = []
section = 1  
for y in range(0, 200, 40):
  for x in range(0, 200, 40):
      feature_names.append(f"sec{section}_mean")
      feature_names.append(f"sec{section}_std")
      section += 1
feature=feature_df[feature_names]
label=feature_df.name

In [ ]:
feature_df.head(20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature,label ,test_size=0.2, random_state=2020)
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
model = GridSearchCV(LogisticRegression(), params)
model.fit(X_train, y_train)

### **Evalutate Model **

In [ ]:
y_pred2 = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

In [ ]:
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_test, y_pred2) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

## Feature extraction with VGG16

In [ ]:
input_shape=(224,224,3)
vgg = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
for layer in vgg.layers:
    layer.trainable = True
vgg.summary()

In [ ]:
feature_list = []
for imgpath in filenames:
    x = load_img(path+"/"+imgpath,target_size=(224,224))
    width, height = x.size
    xx = width*0.2
    yy =0
    w = width*0.9
    h = height*0.50
    im1 =x.crop((xx, yy, w, h))
    im1= np.array (im1)
    a=cv2.resize(im1,(224,224))
    img_array = img_to_array(a)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = imagenet_utils.preprocess_input(img_array)
    features = vgg.predict(img_array)
    feature_list.append(features)
feat_lst = np.reshape(feature_list,(-1,7*7*512))    

In [ ]:
feat_lst.shape

In [ ]:
f_train, f_test, l_train, l_test = train_test_split(feat_lst, labels, test_size=0.2, random_state=2020)
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
model1 = GridSearchCV(LogisticRegression(), params)
model1.fit(f_train, l_train)

In [ ]:
y_pred21 = model1.predict(f_test)
from sklearn.metrics import accuracy_score
accuracy_score(l_test,y_pred21)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix# compute the confusion matrix
confusion_mtx = confusion_matrix(l_test, y_pred21) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from google.colab import files
files.upload()

In [ ]:
im = load_img("/content/IMG_E5750.JPG",target_size=(224,224))
width, height = im.size
xx = width*0.2
yy =0
w = width*0.9
h = height*0.50
    # Cropped image of above dimension 
    # (It will not change orginal image) 
im1 =im.crop((xx, yy, w, h))
im1= np.array (im1) 
im2 = cv2.GaussianBlur(im1, (3, 3), 0)
a = cv2.Canny(im2, 150, 200)
a=cv2.resize(a,(224,224))
b=features_grid(a)
    # Shows theimage in image viewer 
pre= model.predict(b.reshape(1,-1))
print(pre)
plt.imshow(im)

In [ ]:
im = load_img("/content/video-1594976299 06.jpg",target_size=(224,224))
img_array = img_to_array(x)
img_array = np.expand_dims(img_array, axis=0)
features = vgg.predict(img_array)
features_=np.array(features)
pred=model1.predict(features_.reshape(-1,7*7*512))
print(pred)
plt.imshow(im)